In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
JSTOR_STASH

Config,Param,Value
Path,Root Dir,/Users/rj416/.cache/hashstash/osp_jstor
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,13891


In [3]:
PMLA_STASH

Config,Param,Value
Path,Root Dir,/Users/rj416/.cache/hashstash/osp_pmla
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,7351


In [4]:
df_pmla = get_pmla_df().set_index('id')

100%|██████████| 71902/71902 [00:03<00:00, 19376.80it/s]


In [5]:
df_jstor = get_jstor_data().set_index('id')

In [6]:
import random

def collect_tree_stats(stash, limit=None):
    all_tree_stats = []
    for id,docstr in tqdm(stash.items(),total=len(stash) if limit is None else limit):
        doc = stanza.Document.from_serialized(docstr)
        tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
        tree_stats['id'] = id
        all_tree_stats.append(tree_stats)
        if limit is not None and len(all_tree_stats) >= limit:
            break
    all_tree_stats = pd.concat(all_tree_stats)
    for col in all_tree_stats.columns:
        if col not in ['id','num_words','sent']:
            all_tree_stats[col+'_per_100words'] = all_tree_stats[col] / all_tree_stats['num_words'] * 100
    return all_tree_stats

In [7]:
def get_pmla_tree_df(limit=None):
    df_pmla = get_pmla_df().set_index('id')
    all_tree_stats = collect_tree_stats(PMLA_STASH, limit=limit)
    df_all_tree_stats = all_tree_stats.merge(df_pmla, on='id', how='left')
    return df_all_tree_stats

In [8]:
df_pmla_tree = get_pmla_tree_df()

100%|██████████| 7351/7351 [1:00:59<00:00,  2.01it/s]


In [9]:
def get_jstor_tree_df(limit=None):
    df_jstor = get_jstor_data().set_index('id')
    all_tree_stats = collect_tree_stats(JSTOR_STASH, limit=limit)
    df_all_tree_stats = all_tree_stats.merge(df_jstor, on='id', how='left')
    return df_all_tree_stats

In [10]:
df_jstor_tree = get_jstor_tree_df(limit=None)

100%|██████████| 14071/14071 [2:13:10<00:00,  1.76it/s]  


In [11]:
df_combined = pd.concat([df_pmla_tree.assign(discipline='Literature'), df_jstor_tree.assign(discipline='Philosophy')])



In [15]:
import langdetect

LANG_STASH = HashStash('osp_lang')


def get_lang(sentstr):
    lang = LANG_STASH.get(sentstr)
    if lang:
        return lang
    else:
        try:
            lang = langdetect.detect(sentstr)
            if not lang: lang = 'unknown'
            LANG_STASH[sentstr] = lang
            return lang
        except:
            return 'unknown'


In [16]:
df_combined['lang'] = [get_lang(sent) for sent in tqdm(df_combined['sent'])]

100%|██████████| 5957584/5957584 [3:05:13<00:00, 536.05it/s]  


In [17]:
def has_digits(sentstr):
    return any(c.isdigit() for c in sentstr)
df_combined['has_digits'] = [has_digits(sent) for sent in tqdm(df_combined['sent'])]

100%|██████████| 5957584/5957584 [00:22<00:00, 261655.82it/s]


In [18]:
df_combined.to_pickle('../data/raw/df_sents_combined.pkl.gz')

In [20]:
!open ../data/raw